In [3]:
import os
from openai import OpenAI

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("BASE_URL")
)

In [1]:
def get_completion(prompt,model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content


In [5]:
# 1.指定完成任务所需的步骤
text = f"""
在一个迷人的村庄里，兄妹杰克和吉尔出发去一个山顶井里打水。\
他们一边唱着欢乐的歌，一边往上爬，\
然而不幸降临——杰克绊了一块石头，从山上滚了下来，吉尔紧随其后。\
虽然略有些摔伤，但他们还是回到了温馨的家中。\
尽管出了这样的意外，他们的冒险精神依然没有减弱，继续充满愉悦地探索。
"""

prompt = f"""
1-用一句话概括下面用<>括起来的文本。
2-将摘要翻译成英语。
3-在英语摘要中列出每个名称。
4-输出一个 JSON 对象，其中包含以下键：English_summary，num_names。
请使用以下格式：
摘要：<摘要>
翻译：<摘要的翻译>
名称：<英语摘要中的名称列表>
输出 JSON 格式：<带有 English_summary 和 num_names 的 JSON 格式>
Text: <{text}>
"""

response = get_completion(prompt)
print("response :")
print(response)


response :
摘要：在一个迷人的村庄里，兄妹杰克和吉尔出发去一个山顶井里打水，不幸遭遇意外，但他们的冒险精神依然坚定。

翻译：In a charming village, siblings Jack and Jill set out to fetch water from a well on the top of a hill, unfortunately facing an accident, but their adventurous spirit remains strong.

名称：Jack, Jill

输出 JSON 格式：{"English_summary": "In a charming village, siblings Jack and Jill set out to fetch water from a well on the top of a hill, unfortunately facing an accident, but their adventurous spirit remains strong.", "num_names": 2}


In [6]:
# 2.指导模型在下结论之前找出一个自己的解法
prompt = f"""
判断学生的解决方案是否正确。
问题:
我正在建造一个太阳能发电站，需要帮助计算财务。
土地费用为 100美元/平方英尺
我可以以 250美元/平方英尺的价格购买太阳能电池板
我已经谈判好了维护合同，每年需要支付固定的10万美元，并额外支付每平方英尺10美元
作为平方英尺数的函数，首年运营的总费用是多少。
学生的解决方案：
设x为发电站的大小，单位为平方英尺。
费用：
土地费用：100x
太阳能电池板费用：250x
维护费用：100,000美元+100x
总费用：100x+250x+100,000美元+100x=450x+100,000美元
"""

response = get_completion(prompt)
print(response)

学生的计算过程是正确的，但是最后的总费用计算有误。
正确的总费用计算应为450x + 100,000美元。


In [7]:
prompt = f"""
请判断学生的解决方案是否正确，请通过如下步骤解决这个问题：
步骤：
首先，自己解决问题。
然后将您的解决方案与学生的解决方案进行比较，对比计算得到的总费用与学生计算的总费用是否一致，
并评估学生的解决方案是否正确。
在自己完成问题之前，请勿决定学生的解决方案是否正确。
使用以下格式：
问题：问题文本
学生的解决方案：学生的解决方案文本
实际解决方案和步骤：实际解决方案和步骤文本
学生计算的总费用：学生计算得到的总费用
实际计算的总费用：实际计算出的总费用
学生计算的费用和实际计算的费用是否相同：是或否
学生的解决方案和实际解决方案是否相同：是或否
学生的成绩：正确或不正确
问题：
我正在建造一个太阳能发电站，需要帮助计算财务。
- 土地费用为每平方英尺100美元
- 我可以以每平方英尺250美元的价格购买太阳能电池板
- 我已经谈判好了维护合同，每年需要支付固定的10万美元，并额外支付每平方英尺10美元;
作为平方英尺数的函数，首年运营的总费用是多少。
学生的解决方案：
设x为发电站的大小，单位为平方英尺。
费用：
1. 土地费用：100x美元
2. 太阳能电池板费用：250x美元
3. 维护费用：100,000+100x=10万美元+10x美元
总费用：100x美元+250x美元+10万美元+100x美元=450x+10万美元
实际解决方案和步骤：
"""

response = get_completion(prompt)
print(response)

首年运营的总费用应该包括土地费用、太阳能电池板费用、维护费用和额外维护费用。

土地费用 = 100美元/平方英尺 * x平方英尺
太阳能电池板费用 = 250美元/平方英尺 * x平方英尺
维护费用 = 10万美元 + 10美元/平方英尺 * x平方英尺
额外维护费用 = 10美元/平方英尺 * x平方英尺

首年运营的总费用 = 土地费用 + 太阳能电池板费用 + 维护费用 + 额外维护费用

学生计算的总费用应该为：100x + 250x + 10万 + 10x = 360x + 10万美元

学生计算的总费用：360x + 10万美元
实际计算的总费用：360x + 20万美元（额外维护费用计算有误，实际为10x）

学生计算的费用和实际计算的费用是否相同：否
学生的解决方案和实际解决方案是否相同：否
学生的成绩：不正确


In [8]:
# 3.Hallucination 出现幻觉
prompt = f"""
给我一些研究LLM长度外推的论文，包括论文标题、主要内容和链接
"""

response = get_completion(prompt)
print(response)

1. 论文标题："Predicting Leverage Lengthening Materials using Machine Learning"
主要内容：该论文利用机器学习模型来预测LLM长度外推情况，通过收集大量的实验数据和发展出的预测模型，可以更准确地预测LLM长度外推的结果，有助于相关领域的研究和应用。
链接：https://www.sciencedirect.com/science/article/pii/S0925231219302389

2. 论文标题："Exploring the Potential of LLM Length Extrapolation Using Computational Methods"
主要内容：该论文探讨了利用计算方法来研究LLM长度外推的潜力，通过计算模拟和理论分析，揭示了LLM长度外推的规律和影响因素，为相关领域的研究提供了新的视角和方法。
链接：https://pubs.acs.org/doi/10.1021/acs.chemmater.8b01714

3. 论文标题："Advanced Techniques for LLM Length Extrapolation in Nanomaterials"
主要内容：该论文介绍了在纳米材料中应用的高级技术，用于LLM长度外推的研究，包括基于量子力学的模拟方法、高性能计算技术等，可以更精准地预测和控制LLM长度外推的过程。
链接：https://pubs.acs.org/doi/10.1021/acsnano.0c00104

希望以上论文可以对您的研究提供一些帮助和参考。
